In [3]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd 

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
    
class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df

,行政區域,總數量,可借數量,可還數量,時間
0,大安區,28,9,19,2024-07-03 15:40:34
1,大安區,21,0,21,2024-07-03 15:40:28
2,大安區,16,11,5,2024-07-03 15:40:34
3,大安區,11,1,10,2024-07-03 15:25:22
4,大安區,16,1,15,2024-07-03 15:32:08
...,...,...,...,...,...
1425,臺大公館校區,30,15,0,2024-07-03 15:21:15
1426,臺大公館校區,20,20,0,2024-07-03 15:38:14
1427,臺大公館校區,24,14,4,2024-07-03 15:37:19
1428,臺大公館校區,40,30,0,2024-07-03 15:40:34


In [4]:
df1 = df.set_index(keys=["行政區域"])
df1.groupby(level='行政區域')[['總數量','可借數量','可還數量']].sum()

,總數量,可借數量,可還數量
行政區域,,,
中山區,3819,1427,2362
中正區,3209,1633,1520
信義區,3331,1313,1941
內湖區,3499,1420,2022
北投區,2472,938,1489
南港區,2317,922,1383
士林區,2962,1300,1602
大同區,1630,530,1086
大安區,4987,2053,2904
